In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
w = pd.read_csv(PATH["w.tsv"], sep="\t", index_col=0)

w.columns.name = "Factor"

h = pd.read_csv(PATH["h.tsv"], sep="\t", index_col=0)

h.columns.name = SETTING["sample_alias"]

In [ ]:
w_signature = pd.DataFrame(index=w.index)

for factor_name, factor_values in w.items():

    w_signature.loc[
        ccal.select_series_indices(
            factor_values,
            ">",
            standard_deviation=1,
            title={
                "text": "NMF K={} W {} Signature".format(SETTING["nmf_k"], factor_name)
            },
            xaxis={"title": "Rank"},
            yaxis={"title": "Factor Value"},
        ),
        factor_name,
    ] = 1

w_signature.columns = pd.Index(
    ("{} Signature".format(factor_name) for factor_name in w_signature.columns),
    name="Signature",
)

In [ ]:
h_signature = pd.DataFrame(columns=h.columns)

for factor_name, factor_values in h.iterrows():

    h_signature.loc[
        factor_name,
        ccal.select_series_indices(
            factor_values,
            ">",
            standard_deviation=1,
            title={
                "text": "NMF K={} H {} Signature".format(SETTING["nmf_k"], factor_name)
            },
            xaxis={"title": "Rank"},
            yaxis={"title": "Factor Value"},
        ),
    ] = 1

h_signature.index = pd.Index(
    ("{} Signature".format(factor_name) for factor_name in h_signature.index),
    name="Signature",
)

In [ ]:
if os.path.isfile(PATH["w|cluster_x_column.tsv"]):

    w_element_cluster = pd.read_csv(
        PATH["w|cluster_x_column.tsv"], sep="\t", index_col=0
    ).apply(lambda column: column.argmax())

    w_element_cluster.name = "Cluster"

    w_element_cluster.index.name = w.index.name

else:

    w_element_cluster = None

if os.path.isfile(PATH["h|cluster_x_column.tsv"]):

    h_element_cluster = pd.read_csv(
        PATH["h|cluster_x_column.tsv"], sep="\t", index_col=0
    ).apply(lambda column: column.argmax())

    h_element_cluster.name = "Cluster"

    h_element_cluster.index.name = h.columns.name

else:

    h_element_cluster = None

In [ ]:
feature_summary = pd.concat((w, w_signature, w_element_cluster), axis=1)

feature_summary = ccal.clean_and_write_df_to_tsv(
    feature_summary,
    feature_summary.index.name,
    os.path.join(PATH["summary/"], "feature_x_summary_information.tsv"),
)

feature_summary

In [ ]:
sample_summary = pd.concat((h.T, h_signature.T, h_element_cluster), axis=1)

sample_summary = ccal.clean_and_write_df_to_tsv(
    sample_summary,
    sample_summary.index.name,
    os.path.join(PATH["summary/"], "sample_x_summary_information.tsv"),
)

sample_summary